# Building AI Agents with OCI and Llama Stack

This notebook demonstrates how to build Agents using Oracle Cloud Infrastructure (OCI) Generative AI models through Llama Stack.

## What You'll Learn

1. **Basic Agents** - Create agents with custom instructions
2. **Multi-Turn Conversations** - Agents that maintain context across interactions
3. **Agentic Workflows** - Chain multiple agent capabilities together

## Prerequisites

1. **Install required packages:**
   ```bash
   pip install llama-stack-client oci
   ```

2. **Configure OCI credentials:**
   - Set up `~/.oci/config` with your OCI credentials
   - Set the `OCI_COMPARTMENT_OCID` environment variable
   - Set the `OCI_REGION` environment variable
   - Set the `OCI_AUTH_TYPE` environment variable with 'config_file'

3. **Start Llama Stack server with OCI provider:**
   ```bash
   llama stack run /path/to/your_oci_config.yaml
   ```

## Setup and Configuration

In [1]:
# Make sure that your llama stack client is compatible with your llama stack server
# Optional in case you need to select a specific venv enviornment.
import sys
sys.path.insert(0, '/Users/omara/oci/venv/lib/python3.12/site-packages')
print(f"Python path updated to use venv")

Python path updated to use venv


In [2]:
# Import required libraries
from llama_stack_client import LlamaStackClient, Agent
import os
import json

# Check environment
if not os.getenv("OCI_COMPARTMENT_OCID"):
    print("⚠️  WARNING: OCI_COMPARTMENT_OCID environment variable not set")
    print("Please set it with: export OCI_COMPARTMENT_OCID='ocid1.compartment.oc1..xxx'")
else:
    print("✅ OCI_COMPARTMENT_OCID is set")

# Initialize client
client = LlamaStackClient(base_url="http://localhost:8321")
print("✅ Connected to Llama Stack server")

# Get available models
models = client.models.list()
print(f"\n📋 Available models:")
for model in models:
    print(f"   • {model.id}")

# Select primary model
model_id = models[0].id
print(f"\n Using model: {model_id}")

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/models "HTTP/1.1 200 OK"


✅ OCI_COMPARTMENT_OCID is set
✅ Connected to Llama Stack server

📋 Available models:
   • oci/google.gemini-2.5-flash
   • oci/google.gemini-2.5-pro
   • oci/google.gemini-2.5-flash-lite
   • oci/xai.grok-4-fast-non-reasoning
   • oci/xai.grok-4-fast-reasoning
   • oci/xai.grok-code-fast-1
   • oci/xai.grok-4
   • oci/xai.grok-3-mini-fast
   • oci/xai.grok-3-fast
   • oci/xai.grok-3
   • oci/xai.grok-3-mini

 Using model: oci/google.gemini-2.5-flash


---
## 1. Creating a Basic Agent

Agents in Llama Stack are autonomous AI systems with:
- **Instructions**: System-level guidance that shapes behavior
- **Sessions**: Conversation contexts that maintain history
- **Turns**: Individual interactions within a session

In [3]:
# Create a specialized agent with custom instructions
cloud_expert_agent = Agent(
    client=client,
    model=model_id,
    instructions="""You are an Oracle Cloud Infrastructure (OCI) expert assistant.
You help users understand and work with OCI services including:
- Compute instances and shapes
- Networking (VCN, subnets, security lists)
- Storage (Block, Object, File)
- AI/ML services and GPU instances
- Identity and Access Management (IAM)

Always provide practical, actionable advice with examples when possible.
If you're unsure about something, say so clearly.""",
)

print("✅ Created OCI Expert Agent")

✅ Created OCI Expert Agent


In [4]:
# Create a session for our agent
session_id = cloud_expert_agent.create_session(session_name="oci_consultation")
print(f"✅ Created session: {session_id}")

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/conversations "HTTP/1.1 200 OK"


✅ Created session: conv_00838b9921668e9fc1e24431c2e749732d041b7a4e0af149


In [5]:
# Helper functions for chatting with agents and models

def chat_with_agent(agent, session_id, message, stream=True, verbose=False):
    """
    Send a message to an agent and get the full response.
    Properly handles both text responses and tool calls.
    """
    print(f"\n👤 User: {message}\n")
    print("🤖 Agent: ", end='')

    response = agent.create_turn(
        session_id=session_id,
        messages=[{"role": "user", "content": message}],
        stream=stream,
    )

    streamed_text = ""
    tool_calls_made = []

    for chunk in response:
        event = chunk.event
        event_type = event.event_type

        if verbose:
            print(f"\n[DEBUG] Event type: {event_type}")
            print(f"[DEBUG] Event: {event}")

        if event_type == "step_started":
            # Check if this is a tool execution step
            if hasattr(event, 'step_type'):
                if event.step_type == "tool_execution":
                    print("\n    Executing tool...", end='')

        elif event_type == "step_progress":
            # Extract text from step_progress events
            if hasattr(event, 'delta'):
                if hasattr(event.delta, 'text') and event.delta.text:
                    text = event.delta.text
                    streamed_text += text
                    print(text, end='', flush=True)

        elif event_type == "step_completed":
            # Check for tool call results
            if hasattr(event, 'step_details'):
                step = event.step_details
                if hasattr(step, 'tool_calls') and step.tool_calls:
                    for tc in step.tool_calls:
                        tool_name = tc.tool_name if hasattr(tc, 'tool_name') else 'unknown'
                        tool_calls_made.append(tool_name)
                        print(f"\n   ✅ Tool '{tool_name}' completed")

        elif event_type == "turn_completed":
            # Turn is complete - get final text if available
            if hasattr(event, 'turn') and hasattr(event.turn, 'output_message'):
                output = event.turn.output_message
                if hasattr(output, 'content') and output.content:
                    # Get text from content (may be a list of content blocks)
                    if isinstance(output.content, list):
                        for block in output.content:
                            if hasattr(block, 'text') and block.text:
                                if not streamed_text:  # Only if we haven't streamed it already
                                    print(block.text)
                                    streamed_text = block.text
                    elif hasattr(output.content, 'text'):
                        if not streamed_text:
                            print(output.content.text)
                            streamed_text = output.content.text
            break

    print("\n")

    if tool_calls_made:
        print(f" Tools used: {', '.join(tool_calls_made)}")

    return streamed_text

In [6]:
# Ask the agent a question
response = chat_with_agent(
    cloud_expert_agent,
    session_id,
    "What GPU shapes are available on OCI for AI workloads?"
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/responses "HTTP/1.1 200 OK"



👤 User: What GPU shapes are available on OCI for AI workloads?

🤖 Agent: Oracle Cloud Infrastructure (OCI) offers a robust selection of GPU shapes, primarily from **NVIDIA**, designed to accelerate various AI, Machine Learning, Deep Learning, and High-Performance Computing (HPC) workloads. These instances provide massive parallelism and high memory bandwidth crucial for these demanding tasks.

OCI categorizes its GPU offerings into two main types:

1.  **Virtual Machine (VM) GPU Shapes:** These provide flexibility, easier scaling, and are great for development, smaller to medium-scale training, and inference workloads.
2.  **Bare Metal (BM) GPU Shapes:** These offer direct access to the underlying hardware, maximizing performance by eliminating virtualization overhead. They are ideal for the largest, most demanding training jobs, distributed deep learning, and HPC.

Here's a breakdown of the available GPU shapes for AI workloads on OCI:

---

### I. Virtual Machine (VM) GPU Shapes

VM

In [7]:
# Continue the conversation - the agent maintains context!
response = chat_with_agent(
    cloud_expert_agent,
    session_id,
    "How do I choose between them for training a large language model?"
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/responses "HTTP/1.1 200 OK"



👤 User: How do I choose between them for training a large language model?

🤖 Agent: Choosing the right OCI GPU shape for training a Large Language Model (LLM) is a critical decision that impacts performance, cost, and the feasibility of your project. "Large" can mean anything from fine-tuning a 7B parameter model to pre-training a 100B+ parameter model from scratch.

Here's a breakdown of how to think about it, considering the unique demands of LLM training:

### Key Considerations for LLM Training

1.  **VRAM (GPU Memory) is Paramount:**
    *   LLMs are memory hogs. Parameters, activations, gradients, and optimizer states all reside in VRAM.
    *   The model "fitting" in memory is the first and foremost requirement. If it doesn't fit, you can't train it efficiently (or at all without complex offloading).
    *   The optimizer choice (e.g., AdamW) significantly impacts memory usage.
    *   Techniques like 8-bit or 4-bit quantization can reduce memory footprint but might affect prec

---
## 2. Specialized Agent Personas

You can create multiple agents with different expertise for different tasks.

In [8]:
# Create a security-focused agent
security_agent = Agent(
    client=client,
    model=model_id,
    instructions="""You are an OCI Security Specialist. Your expertise includes:
- Identity and Access Management (IAM) policies
- Network security (Security Lists, NSGs, WAF)
- Encryption (Vault, Key Management)
- Security Zones and Cloud Guard
- Compliance and auditing

Always emphasize security best practices and zero-trust principles.
Warn users about potential security risks in their configurations.
Provide specific policy examples when relevant.""",
)

# Create a DevOps agent
devops_agent = Agent(
    client=client,
    model=model_id,
    instructions="""You are an OCI DevOps Engineer. Your expertise includes:
- CI/CD pipelines with OCI DevOps
- Container services (OKE, Container Instances)
- Infrastructure as Code (Terraform, Resource Manager)
- Monitoring and observability
- Automation and scripting with OCI CLI/SDK

Provide practical, automation-focused advice.
Include code snippets and CLI commands when helpful.
Emphasize repeatability and infrastructure as code.""",
)

print("✅ Created specialized agents:")
print("   • Security Agent")
print("   • DevOps Agent")

✅ Created specialized agents:
   • Security Agent
   • DevOps Agent


In [9]:
# Create sessions for each agent
security_session = security_agent.create_session(session_name="security_review")
devops_session = devops_agent.create_session(session_name="devops_planning")

print("✅ Sessions created")

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/conversations "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/conversations "HTTP/1.1 200 OK"


✅ Sessions created


In [10]:
# Ask the security agent
print("=" * 80)
print("SECURITY AGENT")
print("=" * 80)
response = chat_with_agent(
    security_agent,
    security_session,
    "What IAM policies do I need to set up for a team that manages AI workloads?"
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/responses "HTTP/1.1 200 OK"


SECURITY AGENT

👤 User: What IAM policies do I need to set up for a team that manages AI workloads?

🤖 Agent: Managing AI workloads in OCI involves a diverse set of services and requires carefully crafted IAM policies to ensure security, compliance, and optimal operational efficiency. As an OCI Security Specialist, I strongly advocate for a **zero-trust approach**, granting the **least privilege necessary** for each role and service.

Here's a breakdown of recommended IAM policies, structured by typical AI team roles and OCI services, along with essential security best practices.

## Core Principles for AI Workload IAM Policies

1.  **Zero Trust**: Never trust, always verify. Grant the minimum permissions required for a role or service to perform its function.
2.  **Compartment-Based Isolation**: Organize your OCI resources into logical compartments (e.g., `AI_Dev`, `AI_Test`, `AI_Prod`, `AI_DataStores`). Policies should primarily be scoped to these compartments.
3.  **Role-Based Acces

In [11]:
# Ask the DevOps agent the same general area but different focus
print("=" * 80)
print("DEVOPS AGENT")
print("=" * 80)
response = chat_with_agent(
    devops_agent,
    devops_session,
    "How do I set up a CI/CD pipeline for deploying ML models on OCI?"
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/responses "HTTP/1.1 200 OK"


DEVOPS AGENT

👤 User: How do I set up a CI/CD pipeline for deploying ML models on OCI?

🤖 Agent: Setting up a CI/CD pipeline for ML model deployment on OCI involves several key services working together. The goal is to automate the process from model training (or model artifact generation) to model serving in a reliable, repeatable, and scalable manner.

As an OCI DevOps Engineer, I strongly advocate for **Infrastructure as Code (IaC)** using Terraform and leveraging OCI DevOps for pipeline orchestration.

---

### Key OCI Services Involved

1.  **OCI Code Repositories (or GitHub/GitLab):** Stores your ML inference code, Dockerfiles, pipeline definitions, and deployment manifests.
2.  **OCI Data Science (DS):**
    *   **Model Catalog:** Centralized repository for storing, versioning, and managing ML model artifacts. Crucial for tracking model lineage.
    *   **Model Deployments:** Managed service for deploying models as HTTP endpoints. Ideal for quick and easy serving.
3.  **OCI Cont

---
## 3. Multi-Agent Collaboration Pattern

For complex tasks, you can coordinate multiple agents to work together.

In [12]:
def multi_agent_consultation(question: str) -> dict:
    """
    Get perspectives from multiple specialized agents on a question.
    """
    print(f"\n{'='*80}")
    print(f" MULTI-AGENT CONSULTATION")
    print(f"{'='*80}")
    print(f"\n Question: {question}\n")

    responses = {}

    # Get security perspective
    print("\n SECURITY PERSPECTIVE:")
    print("-" * 40)
    sec_session = security_agent.create_session(session_name="consultation_security")
    response = security_agent.create_turn(
        session_id=sec_session,
        messages=[{"role": "user", "content": f"From a security perspective: {question}"}],
        stream=True,
    )
    sec_response = ""
    for chunk in response:
        if chunk.event.event_type == "turn_completed":
            sec_response = chunk.event.final_text
            break
        elif chunk.event.event_type == "step_progress":
            if hasattr(chunk.event.delta, 'text'):
                print(chunk.event.delta.text, end='', flush=True)
    responses['security'] = sec_response
    print("\n")

    # Get DevOps perspective
    print("\n DEVOPS PERSPECTIVE:")
    print("-" * 40)
    devops_sess = devops_agent.create_session(session_name="consultation_devops")
    response = devops_agent.create_turn(
        session_id=devops_sess,
        messages=[{"role": "user", "content": f"From a DevOps perspective: {question}"}],
        stream=True,
    )
    devops_response = ""
    for chunk in response:
        if chunk.event.event_type == "turn_completed":
            devops_response = chunk.event.final_text
            break
        elif chunk.event.event_type == "step_progress":
            if hasattr(chunk.event.delta, 'text'):
                print(chunk.event.delta.text, end='', flush=True)
    responses['devops'] = devops_response
    print("\n")

    return responses

print("✅ Multi-agent consultation function ready")

✅ Multi-agent consultation function ready


In [13]:
# Run a multi-agent consultation
results = multi_agent_consultation(
    "How should I architect a production ML inference service on OCI?"
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/conversations "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/responses "HTTP/1.1 200 OK"



 MULTI-AGENT CONSULTATION

 Question: How should I architect a production ML inference service on OCI?


 SECURITY PERSPECTIVE:
----------------------------------------
Architecting a production ML inference service on OCI from a security perspective requires a robust, multi-layered approach grounded in **Zero-Trust principles**. Assume compromise at every layer and implement controls to mitigate risks.

Here's a comprehensive architecture breakdown, emphasizing security best practices:

## I. Core Architecture Components (Conceptual View)

1.  **Client Application:** External/Internal users or applications making inference requests.
2.  **Web Application Firewall (WAF) / DDoS Protection:** OCI WAF provides L7 protection for public endpoints.
3.  **API Gateway:** Serves as the single, secure entry point for inference requests.
4.  **Load Balancer:** Distributes traffic across inference endpoints.
5.  **ML Inference Compute:** The actual service hosting the trained model.
    *   **OCI

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/conversations "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/responses "HTTP/1.1 200 OK"





 DEVOPS PERSPECTIVE:
----------------------------------------
As an OCI DevOps Engineer, architecting a production ML inference service on OCI demands a focus on **automation, repeatability, scalability, and observability**. We want to treat our ML models and their serving infrastructure as code, integrate them into CI/CD pipelines, and ensure robust monitoring.

Here's a comprehensive approach, emphasizing OCI services and DevOps best practices.

## Core Principles for Production ML Inference

1.  **Infrastructure as Code (IaC):** Provision and manage all infrastructure with Terraform via OCI Resource Manager. This ensures repeatability and version control.
2.  **Containerization:** Package your ML model and its serving logic into Docker containers. This provides environment consistency.
3.  **CI/CD:** Automate the build, test, and deployment of your model and its inference service using OCI DevOps.
4.  **Model Registry:** Store, version, and manage your trained models securely.
5.

---
## 4. Agentic Workflow: Infrastructure Review

Let's add an architect agent that reviews the responses of the security and devops agents

In [14]:
# Create an architect agent that synthesizes recommendations
architect_agent = Agent(
    client=client,
    model=model_id,
    instructions="""You are a Senior OCI Solutions Architect. Your role is to:
- Review infrastructure requirements and constraints
- Synthesize input from security and DevOps perspectives
- Provide comprehensive, balanced architectural recommendations
- Consider cost, performance, security, and operational aspects
- Create actionable implementation plans

You provide executive-level summaries followed by detailed technical guidance.""",
)

print("✅ Created Architect Agent")

✅ Created Architect Agent


In [15]:
def infrastructure_review_workflow(requirements: str):
    """
    A multi-stage workflow for infrastructure review.

    Stage 1: Security Analysis
    Stage 2: DevOps Analysis
    Stage 3: Architecture Synthesis
    """
    print("\n" + "="*80)
    print("  INFRASTRUCTURE REVIEW WORKFLOW")
    print("="*80)
    print(f"\n Requirements:\n{requirements}\n")

    # Stage 1: Security Analysis
    print("\n" + "="*80)
    print(" STAGE 1: Security Analysis")
    print("="*80)
    sec_session = security_agent.create_session(session_name="review_security")
    sec_response = security_agent.create_turn(
        session_id=sec_session,
        messages=[{"role": "user", "content": f"Review these requirements for security considerations:\n\n{requirements}\n\nProvide key security requirements and concerns."}],
        stream=True,
    )
    security_analysis = ""
    for chunk in sec_response:
        if chunk.event.event_type == "turn_completed":
            security_analysis = chunk.event.final_text
            break
        elif chunk.event.event_type == "step_progress":
            if hasattr(chunk.event.delta, 'text'):
                print(chunk.event.delta.text, end='', flush=True)
    print("\n")

    # Stage 2: DevOps Analysis
    print("\n" + "="*80)
    print(" STAGE 2: DevOps Analysis")
    print("="*80)
    devops_sess = devops_agent.create_session(session_name="review_devops")
    devops_response = devops_agent.create_turn(
        session_id=devops_sess,
        messages=[{"role": "user", "content": f"Review these requirements for DevOps implementation:\n\n{requirements}\n\nProvide key operational requirements and automation opportunities."}],
        stream=True,
    )
    devops_analysis = ""
    for chunk in devops_response:
        if chunk.event.event_type == "turn_completed":
            devops_analysis = chunk.event.final_text
            break
        elif chunk.event.event_type == "step_progress":
            if hasattr(chunk.event.delta, 'text'):
                print(chunk.event.delta.text, end='', flush=True)
    print("\n")

    # Stage 3: Architecture Synthesis
    print("\n" + "="*80)
    print(" STAGE 3: Architecture Synthesis")
    print("="*80)
    arch_session = architect_agent.create_session(session_name="review_synthesis")
    synthesis_prompt = f"""Based on the following inputs, provide a comprehensive infrastructure recommendation:

## Original Requirements:
{requirements}

## Security Analysis:
{security_analysis}

## DevOps Analysis:
{devops_analysis}

Provide a synthesized recommendation that addresses all concerns with a clear implementation plan."""

    arch_response = architect_agent.create_turn(
        session_id=arch_session,
        messages=[{"role": "user", "content": synthesis_prompt}],
        stream=True,
    )
    final_recommendation = ""
    for chunk in arch_response:
        if chunk.event.event_type == "turn_completed":
            final_recommendation = chunk.event.final_text
            break
        elif chunk.event.event_type == "step_progress":
            if hasattr(chunk.event.delta, 'text'):
                print(chunk.event.delta.text, end='', flush=True)
    print("\n")

    print("\n" + "="*80)
    print("✅ REVIEW COMPLETE")
    print("="*80)

    return {
        "security_analysis": security_analysis,
        "devops_analysis": devops_analysis,
        "final_recommendation": final_recommendation
    }

print("✅ Infrastructure review workflow ready")

✅ Infrastructure review workflow ready


In [16]:
# Run the infrastructure review workflow
requirements = """
We need to deploy a real-time ML inference service with the following requirements:

1. Handle 1000 requests per second at peak
2. Maximum latency of 100ms for 99th percentile
3. Use GPU-accelerated inference
4. Support model updates without downtime
5. Store inference logs for 90 days
6. Operate in us-phoenix-1 region
7. Budget: ~$10,000/month
"""

review_results = infrastructure_review_workflow(requirements)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/conversations "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/responses "HTTP/1.1 200 OK"



  INFRASTRUCTURE REVIEW WORKFLOW

 Requirements:

We need to deploy a real-time ML inference service with the following requirements:

1. Handle 1000 requests per second at peak
2. Maximum latency of 100ms for 99th percentile
3. Use GPU-accelerated inference
4. Support model updates without downtime
5. Store inference logs for 90 days
6. Operate in us-phoenix-1 region
7. Budget: ~$10,000/month



 STAGE 1: Security Analysis
As an OCI Security Specialist, I've reviewed your requirements for a real-time ML inference service. Deploying a high-performance, real-time system like this, especially with GPU acceleration and continuous updates, introduces several critical security considerations. Our focus will be on embedding security from design through operations, adhering to **Zero-Trust principles** and **least privilege**, and utilizing OCI's robust security capabilities.

Here are the key security requirements and concerns, categorized for clarity:

---

### Core Security Principles

Be

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/conversations "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/responses "HTTP/1.1 200 OK"





 STAGE 2: DevOps Analysis
This is an excellent set of requirements for a real-time ML inference service, highlighting critical concerns like performance, availability, cost, and maintainability. As an OCI DevOps Engineer, my advice will focus on leveraging OCI services, Infrastructure as Code (IaC), and CI/CD best practices to meet these goals sustainably.

---

## Executive Summary: Recommended OCI Architecture

The core architecture will revolve around **OCI Container Engine for Kubernetes (OKE)** running GPU-enabled instances, integrated with **OCI DevOps** for CI/CD, **OCI Load Balancer** for external access, and robust **OCI Monitoring** and **Logging** for observability. All infrastructure will be defined and managed using **Terraform** via **OCI Resource Manager**.

---

## Detailed Requirements Breakdown & OCI Solutions

### 1. Handle 1000 requests per second at peak

*   **OCI Solution:** **Oracle Container Engine for Kubernetes (OKE)** with **Horizontal Pod Autoscaling (HP

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/conversations "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:8321/v1/responses "HTTP/1.1 200 OK"





 STAGE 3: Architecture Synthesis
As a Senior OCI Solutions Architect, I've reviewed your requirements and the detailed analyses from both Security and DevOps teams. The need for real-time ML inference at scale, with strict latency, high availability, robust security, and cost efficiency, dictates a sophisticated yet agile architecture.

---

## Executive Summary: Real-Time ML Inference on OCI

Our recommended architecture leverages Oracle Cloud Infrastructure's native services to deliver a performant, secure, scalable, and cost-effective real-time ML inference platform. The core will be **Oracle Container Engine for Kubernetes (OKE)** running on **GPU-enabled instances**, providing high-throughput and low-latency inference. This will be protected by an **OCI Web Application Firewall (WAF)** and **Load Balancer**, and managed end-to-end through **OCI DevOps** for continuous integration and delivery.

All infrastructure components will be defined and managed as **Infrastructure as Cod